In [273]:
from xbbg import blp
import pandas as pd
con = pdblp.BCon(timeout=10000)
con.start()
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [331]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]
    tmp = BDP["undl_spot_ticker"] + " Index"
    tmp = blp.bdp(tickers=tmp, flds="country_iso").loc[tmp,:].iloc[:,0]
    BDP["risk"] = list(tmp)

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["start_date"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["ES","NQ","RTY","VG","GX","Z ","PT","XP","TP","HI","XU","IH","TWT"]
# L= ["ES","NQ"]
YK = ["Index"] * len(L)

print("Equity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Equity Futures
(13, 11)


,ticker,name,crncy,exch,exch_code,undl,risk,contract_value,volume,vola,start_date
0,ES,S&P500 EMINI FUT Jun23,USD,US,CME,SPX,US,"214,775","1,690,007",11.95,-
1,NQ,NASDAQ 100 E-MINI Jun23,USD,US,CME,NDX,US,"290,110","661,100",18.14,-
2,RTY,E-Mini Russ 2000 Jun23,USD,US,CME,RTY,US,"94,095","227,154",23.71,20170731
3,VG,EURO STOXX 50 Jun23,EUR,DE,EUX,SX5E,EU,"43,090","756,899",15.77,-
4,GX,DAX INDEX FUTURE Jun23,EUR,DE,EUX,DAX,DE,"400,475","57,807",14.86,-
5,Z,FTSE 100 IDX FUT Jun23,GBP,GB,ICF,UKX,GB,"76,195","101,177",12.57,-
6,PT,S&P/TSX 60 IX FUT Jun23,CAD,CA,MSE,SPTSX60,CA,"239,340","22,740",13.42,-
7,XP,SPI 200 FUTURES Jun23,AUD,AU,SFE,AS51,AU,"178,200","46,807",11.59,20000531
8,TP,TOPIX INDX FUTR Sep23,JPY,JP,OSE,TPX,JP,"22,205,000","102,586",14.98,-
9,HI,HANG SENG IDX FUT Jun23,HKD,HK,HKG,HSI,HK,"966,950","125,048",24.17,-


In [338]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","fut_ctd_isin"]).loc[T,:]


    isin = BDP.loc[T,["fut_ctd_isin"]].dropna(how='all')
    isin = [i for i in isin["fut_ctd_isin"]]

    tmp0 = ["year","dur","yld"]
    tmp1 = ["mty_years","dur_adj_bid","yld_ytm_bid"]


    BDP1 = blp.bdp(tickers = [i+" Corp" for i in isin], flds=tmp1)
    

    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP["fut_ctd_isin"]==i,tmp0[j]] = BDP1.loc[i+" Corp",tmp1[j]]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    for j in ["contract_value","volume"]:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["year","dur","yld","vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20210101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["start_date"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L = ["TU","FV","TY","US","WN","DU","OE","RX","UB","OAT","IK","G ","CN","XM","JB"]
# L = ["TU","FV"]
YK = ["Comdty"] * len(L)

print("Bond Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Bond Futures
(15, 13)


,ticker,name,crncy,exch,exch_code,fut_ctd_isin,year,dur,yld,contract_value,volume,vola,start_date
0,TU,US 2YR NOTE (CBT) Sep23,USD,US,CBT,US91282CEU18,2.01,1.89,4.55,"205,148","830,505",2.17,-
1,FV,US 5YR NOTE (CBT) Sep23,USD,US,CBT,US91282CFZ95,4.47,4.06,3.95,"108,461","1,516,008",4.71,-
2,TY,US 10YR NOTE (CBT)Sep23,USD,US,CBT,US91282CGS44,6.80,5.93,3.82,"113,625","1,899,199",6.19,-
3,US,US LONG BOND(CBT) Sep23,USD,US,CBT,US912810QC53,16.18,11.47,3.86,"127,375","374,177",9.85,-
4,WN,US ULTRA BOND CBT Sep23,USD,US,CBT,US912810SE91,25.43,16.37,4.03,"135,719","295,604",11.19,20100129
5,DU,EURO-SCHATZ FUT Sep23,EUR,DE,EUX,DE000BU22015,2.00,1.91,2.89,"105,410","532,883",1.59,-
6,OE,EURO-BOBL FUTURE Sep23,EUR,DE,EUX,DE000BU25000,4.84,4.49,2.42,"116,540","617,296",4.48,-
7,RX,EURO-BUND FUTURE Sep23,EUR,DE,EUX,DE0001102606,9.17,8.22,2.36,"133,740","740,054",7.85,-
8,UB,EURO-BUXL 30Y BND Sep23,EUR,DE,EUX,DE0001102432,25.17,20.36,2.49,"134,980","87,536",18.05,-
9,OAT,Euro-OAT Future Sep23,EUR,DE,EUX,FR001400H7V7,9.95,8.49,2.95,"128,130","200,359",8.11,20120430


In [341]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["start_date"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["CL","CO","GC"]
YK = ["Comdty"] * len(L)

print("Commodity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Commodity Futures
(3, 10)


,ticker,name,crncy,exch,exch_code,undl,contract_value,volume,vola,start_date
0,CL,WTI CRUDE FUTURE Jul23,USD,US,NYM,USCRWTIC,"70,830","302,563",23.93,-
1,CO,BRENT CRUDE FUTR Aug23,USD,GB,ICE,COY,"75,490","244,222",29.86,-
2,GC,GOLD 100 OZ FUTR Aug23,USD,US,CMX,NaN,"197,900","139,805",12.90,-


In [358]:
def BDP(L):
    T = ["CSBS" + i + " TMUQ Curncy" for i in L]  #Mitsubishi TTM
    tmp = ["px_last","volatility_60d","rsi_30d"]
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, "currency", LS_EXJ )
    BDP.insert(1, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_60d":"vola"})

    return BDP

L = ["USD","EUR","GBP","CAD","AUD","NZD","CHF","NOK","SEK","HKD","CNY"]

print("FX")
BDP = BDP(L)
print(BDP.shape)
BDP

FX
(11, 5)


,currency,ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,139.15,10.35,56.49
1,EUR,CSBSEUR TMUQ Curncy,150.00,11.15,56.29
2,GBP,CSBSGBP TMUQ Curncy,174.69,10.27,62.89
3,CAD,CSBSCAD TMUQ Curncy,104.12,11.06,59.60
4,AUD,CSBSAUD TMUQ Curncy,93.43,12.52,57.68
5,NZD,CSBSNZD TMUQ Curncy,84.84,13.05,51.34
6,CHF,CSBSCHF TMUQ Curncy,154.65,10.74,60.66
7,NOK,CSBSNOK TMUQ Curncy,12.75,13.84,49.90
8,SEK,CSBSSEK TMUQ Curncy,12.87,13.24,47.85
9,HKD,CSBSHKD TMUQ Curncy,17.76,10.28,56.90
